In [3]:
from pathlib import Path

from pydub import AudioSegment

# from google.cloud import speech
# from pydub import AudioSegment

# Load MP4 and get original properties

working_dir = Path.home() / "Desktop/wouter_video/testing_backup"
audio_file_path = working_dir /  "dt_short_audio.mp4"

if not audio_file_path.exists():
    raise FileNotFoundError(f"File not found: {audio_file_path}")

audio = AudioSegment.from_file(audio_file_path, format="mp4")

# # Take a sample chunk (first 30 seconds for language detection)
# sample_duration_ms = 30000  # 30 seconds
# audio_sample = audio[:sample_duration_ms]

# # Convert to mono but keep original frame rate
# audio_sample = audio_sample.set_channels(1)  # Keep original frame rate
# original_frame_rate = audio_sample.frame_rate
# wav_buffer = io.BytesIO()
# audio_sample.export(wav_buffer, format="wav")
# wav_data = wav_buffer.getvalue()

# # Send to Google Cloud Speech with original frame rate
# client = speech.SpeechClient()
# audio_gcs = speech.RecognitionAudio(content=wav_data)
# config = speech.RecognitionConfig(
#     language_code="auto",
# )

# response = client.recognize(config=config, audio=audio_gcs)

# if response.results:
#     detected_language = response.results[0].language_code
#     confidence = response.results[0].alternatives[0].confidence if response.results[0].alternatives else 0.0
#     print(f"Detected language: {detected_language}")
#     print(f"Confidence: {confidence}")
#     print(f"Used sample rate: {original_frame_rate} Hz")
# else:
#     print("No speech detected in sample")

In [8]:
import io

from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech

# Set your project ID
PROJECT_ID = "tnh-scholar"  # Replace with your actual project ID


# Take a sample chunk (30 seconds for testing)
sample_duration_ms = 5000
start_time = 5000
audio_sample = audio[start_time:start_time + sample_duration_ms]

# Export to bytes
wav_buffer = io.BytesIO()
audio_sample.export(wav_buffer, format="wav")
audio_content = wav_buffer.getvalue()

# Create client and config using v2 API
client = SpeechClient()

config = cloud_speech.RecognitionConfig(
    auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
    # language_codes=["en-US", "vi-VN"],  # Note: plural and no "auto"
    model="long",
)

request = cloud_speech.RecognizeRequest(
    recognizer=f"projects/{PROJECT_ID}/locations/global/recognizers/_",
    config=config,
    content=audio_content,
)

# Recognize
response = client.recognize(request=request)

if response.results:
    for result in response.results:
        detected_language = result.language_code if hasattr(result, 'language_code') else "unknown"
        confidence = result.alternatives[0].confidence if result.alternatives else 0.0
        transcript = result.alternatives[0].transcript
        print(f"Language: {detected_language}")
        print(f"Confidence: {confidence}")
        print(f"Transcript: {transcript[:100]}...")
else:
    print("No speech detected")

InvalidArgument: 400 Invalid `language_codes`: field must be non-empty.

In [ ]:
# Quick PoC using your existing Whisper service
from tnh_scholar.audio_processing.transcription_service.whisper_service import WhisperTranscriptionService

# Use a small audio chunk for just language detection
whisper = WhisperTranscriptionService()
result = whisper.transcribe()
detected_language = result.language  # Whisper auto-detects language
print(f"Detected: {detected_language}")